In [2]:
import math
import pandas as pd
import numpy as np

In [196]:
#  dataset having transaction having items per transaction
transactionId=[1,2,3,4,5]
min_support=0.5
confidence=0.75
item_purchased=[['bread','chease','egg','juice'],
                ['bread','chease','juice'],
                ['bread','milk','yogurt'],
                ['bread','juice','milk'],
               ['chease','juice','milk']]
df=pd.DataFrame({'transactionId':transactionId,'item_purchased':item_purchased})

In [250]:
# Calculate unique items and total transactions 
items=set(item for i in range(len(df.item_purchased)) for item in df['item_purchased'][i])
items=list([item] for item in items)
total_transaction=len(transactionId)
print("Total transactions: ",total_transaction,"    items: ",items)

Total transactions:  5     items:  [['juice'], ['chease'], ['milk'], ['bread'], ['yogurt'], ['egg']]


In [167]:
# Calculate items frequency 
def items_frequency(items,item_table):
    count=0
    for j in range(len(item_table)): 
        a=np.array([i in item_table[j] for i in items ])
        if a.all() == True:
            count+=1
    return count

In [168]:
# calculate support for items 
def support(freq):
    return float(freq/total_transaction)

In [169]:
# update item list to combine same featured item in a single item list 
def update_item(items):
    update_item_val=[]
    for i in range(len(items)+1):
        for j in range(i+1,len(items)):
            item=items[i]+items[j]
            update_item_val.append(list(set(item)))       
    return update_item_val

In [212]:
# calculate support  at all levels 
def approri(items):
    df_new=pd.DataFrame(columns=['items','frequency','support'])
    df_list=[]
    while len(df_new) !=2:
        df_new=pd.DataFrame(columns=['items','frequency','support'],index=[])
        for i in range(len(items)):
            current_item=items[i]
            freq=items_frequency(current_item,df.item_purchased)
            supp=support(freq)
            if supp >=min_support: 
                df_new.loc[i]=[current_item,freq,supp]
        print(df_new)
        df_list.append(df_new)
        items=update_item(items)
    return df_list

In [251]:
tables=approri(items)

      items frequency  support
0   [juice]         4      0.8
1  [chease]         3      0.6
2    [milk]         3      0.6
3   [bread]         4      0.8
             items frequency  support
0  [chease, juice]         3      0.6
2   [bread, juice]         3      0.6


In [252]:
# making item and support pair of first table to get individuals support 
table0={str(item_str):support_val for item_str,support_val in zip(tables[0]['items'],tables[0]['support'])}

{"['juice']": 0.8, "['chease']": 0.6, "['milk']": 0.6, "['bread']": 0.8}

In [318]:
# calculate confidence 
def confidence(table0,df_new):
    total_conf=[]
    for l in range(0,len(df_new)):
        conf=[]
        for j in range(len(df_new[l][0])):
            confidence_val=df_new[l][2]/table0[  str(  [df_new[l][0][j]]  )  ]
            conf.append(confidence_val)
        for item,conf in zip(df_new[l][0],conf):
            total_conf.append((item,conf))
    return total_conf
           

In [319]:
conf=confidence(table0,tables[-1].values)
conf

[('chease', 1.0),
 ('juice', 0.7499999999999999),
 ('bread', 0.7499999999999999),
 ('juice', 0.7499999999999999)]